# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/ashurveer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ashurveer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [6]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

mkdir: data: File exists


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31493    0 31493    0     0   171k      0 --:--:-- --:--:-- --:--:--  171k


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70519    0 70519    0     0   505k      0 --:--:-- --:--:-- --:--:--  506k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 10, relationships: 31)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 10, relationships: 31)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What does Meta refer to in the context of larg...,[The ethics of this space remain diabolically ...,Meta is mentioned as one of the organizations ...,single_hop_specifc_query_synthesizer
1,Who is Simon Willison?,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s Weblog discusses AI developme...,single_hop_specifc_query_synthesizer
2,How is the term 'Citus' relevant in the contex...,[the document includes some of the clearest ex...,The provided context does not include any info...,single_hop_specifc_query_synthesizer
3,2024 ki AI me kya kya hua tha ki sab kuch bada...,[Things we learned about LLMs in 2024 31st Dec...,"2024 me bahut kuch hua AI ke duniya me, jaise ...",single_hop_specifc_query_synthesizer
4,What is Claude 3 and how does it compare to ot...,[punch massively above their weight. I run Lla...,Claude 3 is mentioned as a model from Anthropi...,single_hop_specifc_query_synthesizer
5,How does understanding user interaction and th...,[<1-hop>\n\nways we should not be using genera...,Understanding user interaction and the capabil...,multi_hop_abstract_query_synthesizer
6,can I run LLMs on my phone and make them bette...,[<1-hop>\n\nThe ethics of this space remain di...,"Yes, the context shows that many LLMs can now ...",multi_hop_abstract_query_synthesizer
7,Considering the significant advancements in La...,[<1-hop>\n\nThings we learned about LLMs in 20...,"The developments in 2024, including the emerge...",multi_hop_abstract_query_synthesizer
8,How does the Claude 3 model exemplify the adva...,[<1-hop>\n\nways we should not be using genera...,The context highlights that models like Claude...,multi_hop_specific_query_synthesizer
9,Considering the detailed explanations of what ...,[<1-hop>\n\nthe document includes some of the ...,The first segment provides an in-depth explana...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/24 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Microsoft Research's role in the devel...,[My blog in 2023 Here’s the sequel to this pos...,"According to the provided context, Microsoft R...",single_hop_specifc_query_synthesizer
1,What is Spanish in LLMs?,[openly licensed ones are still the world’s mo...,The context does not provide a specific explan...,single_hop_specifc_query_synthesizer
2,What is the significance of AI in 2023 accordi...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison states that 2023 was the breakt...,single_hop_specifc_query_synthesizer
3,Who is Tom Scott and why is he important in th...,"[of what LLMs are, how they work and how they ...",Tom Scott is mentioned in the context as a per...,single_hop_specifc_query_synthesizer
4,How does Simon Willison’s 2023 weblog reflect ...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s 2023 weblog highlights that 2...,multi_hop_abstract_query_synthesizer
5,How have advancements in Large Language Models...,[<1-hop>\n\nThings we learned about LLMs in 20...,"In 2024, significant advancements in Large Lan...",multi_hop_abstract_query_synthesizer
6,How does the use of synthetic training data in...,[<1-hop>\n\nways we should not be using genera...,The context explains that synthetic training d...,multi_hop_abstract_query_synthesizer
7,"Considering the concerns about ethics, trainin...",[<1-hop>\n\nthis. LLMs need better criticism A...,The context emphasizes that LLMs face signific...,multi_hop_abstract_query_synthesizer
8,How do ChatGPT and other LLMs demonstrate thei...,[<1-hop>\n\nopenly licensed ones are still the...,The context explains that ChatGPT and other LL...,multi_hop_specific_query_synthesizer
9,Whas is the main diffence between Llama and Ll...,[<1-hop>\n\npunch massively above their weight...,The context indicates that Llama 3.2 is a more...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/ashurveer/Projects/MP_AI/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/ashurveer/Projects/MP_AI/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague and unclear term in AI that lacks a single, widely understood meaning. Generally, it seems to refer to AI systems that can go away and act on your behalf, often envisioned as digital assistants or travel agent-like models. Another common interpretation involves large language models (LLMs) that have access to tools they can use in loops to solve problems. However, despite much discussion and many prototypes, true functional AI agents running in production remain scarce, partly due to challenges such as gullibility—LLMs tend to believe everything they are told, limiting their reliability and usefulness as decision-makers acting autonomously. The concept of agents is still seen as "coming soon," with some seeing it as dependent on achieving artificial general intelligence (AGI).'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'ordinary-island-89' at:
https://smith.langchain.com/o/8b16d96d-7566-4580-b923-510497302c71/datasets/59106d7b-0ff1-4dd5-889e-7b709b43a400/compare?selectedSessions=0c30d6de-10ac-4254-bb07-8102efc4e7aa




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"how GPT-4 is like, and what we learned about G...","Based on the provided context:\n\nGPT-4, relea...",None,"Based on the context, GPT-4 was considered the...",1,1,0,6.708334,e202c7a7-9f57-46a7-9ef2-a5bf0d2d6f11,871c8c43-6ba2-4895-a8dd-78fe946890ca
1,How do Llama 3.2 and Llama 3.3 70b models comp...,Based on the context provided:\n\n- Llama 3.2 ...,None,The context indicates that Llama 3.2 is a smal...,1,1,0,3.527900,ed23450e-3a1e-425f-bb93-6e85d874f410,6ff2b005-4137-464a-9285-33e36ffe1cb8
2,Whas is the main diffence between Llama and Ll...,The main difference between Llama and Llama 3....,None,The context indicates that Llama 3.2 is a more...,1,0,1,2.856690,9e062b9f-be4c-4452-a1ec-4d43acbeaa15,b92b932e-6476-426c-bb55-f0d2e8c5c725
3,How do ChatGPT and other LLMs demonstrate thei...,ChatGPT and other LLMs demonstrate their capab...,None,The context explains that ChatGPT and other LL...,1,0,0,4.786723,aa27845f-e17f-451f-b48e-c9eff2f5ec94,7a4add3d-f396-4ef0-b3b0-84d0d0db3265
4,"Considering the concerns about ethics, trainin...","Based on the context provided, the societal im...",None,The context emphasizes that LLMs face signific...,1,1,0,7.681055,c8efb7a9-51c2-46ae-9ce8-4b09c6d2c7c5,e0232a3b-a6a7-4ed3-876d-15a165918add
5,How does the use of synthetic training data in...,"The context explains that the idea of ""model c...",None,The context explains that synthetic training d...,1,1,0,3.301896,b90d1d6e-d535-48ce-b22d-8fdf09ffac64,c6344e7f-c39f-4f83-8670-5101896a4c40
6,How have advancements in Large Language Models...,Advancements in Large Language Models (LLMs) i...,None,"In 2024, significant advancements in Large Lan...",1,1,0,4.394502,61554441-1dbf-4ec3-87ae-dcc9100a64b6,0b026add-7bb7-454b-9a0a-5042f1b905d6
7,How does Simon Willison’s 2023 weblog reflect ...,Simon Willison’s 2023 weblog reflects the them...,None,Simon Willison’s 2023 weblog highlights that 2...,1,1,0,4.037114,391fc1ee-0145-49ea-8ef6-66e887d3d4f5,25f4557e-a46c-46b6-bb45-28a4a25663f7
8,Who is Tom Scott and why is he important in th...,I don't know.,None,Tom Scott is mentioned in the context as a per...,0,0,0,156.976644,4cf91cd7-b54d-47bf-af15-197c2ff3bab7,e69e78cd-4e42-444e-a30d-7b97828dc6be
9,What is the significance of AI in 2023 accordi...,"According to Simon Willison, 2023 was the brea...",None,Simon Willison states that 2023 was the breakt...,1,0,0,2.494668,52ef066f-86e8-4431-84e9-6113cbd9968e,c49124b3-1f5b-4e19-a635-a7b4e00072ca


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Alright, here’s the lowdown on “agents” straight from the AI streets: The term “agents” is like that mysterious vibe no one can quite pin down. Folks toss it around, but it’s got no single, crystal-clear meaning. Mostly, it splits into two camps:  \n\n1. Think of agents like your travel agent—they act and do stuff for you out there in the wild world.  \n2. Or, it’s more like some fancy LLM running tools in a loop to solve problems under the hood.  \n\nThe kicker? These so-called “agents” are held back by a big “gullibility problem” — they take whatever you throw at ‘em as gospel, which means they get fooled easy. No solid, bulletproof agents cruising around in production yet, just a lotta buzz and prototypes. Some even say true agents might only roll in once we crack AGI (Artificial General Intelligence), because without that, the gullibility beast won’t be tamed.  \n\nSo yeah, “agents” are more like a dream on the horizon, a “coming soon” marquee flashing but no full-length feature y

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'glossy-stretch-14' at:
https://smith.langchain.com/o/8b16d96d-7566-4580-b923-510497302c71/datasets/59106d7b-0ff1-4dd5-889e-7b709b43a400/compare?selectedSessions=78b4b0ce-8e8d-4357-b17a-ec194677647d




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"how GPT-4 is like, and what we learned about G...","Yo, here’s the lowdown on GPT-4 vibes and the ...",None,"Based on the context, GPT-4 was considered the...",1,1,1,10.298487,e202c7a7-9f57-46a7-9ef2-a5bf0d2d6f11,eba63337-beab-4703-a403-130ff18614b8
1,How do Llama 3.2 and Llama 3.3 70b models comp...,"Alright, here’s the lowdown, fresh and fly: Ll...",None,The context indicates that Llama 3.2 is a smal...,1,1,1,6.067664,ed23450e-3a1e-425f-bb93-6e85d874f410,683e3740-0672-4bf3-a357-bc99ddedbf10
2,Whas is the main diffence between Llama and Ll...,"Alright, here’s the lowdown, fresh and fly: Th...",None,The context indicates that Llama 3.2 is a more...,1,1,1,8.792265,9e062b9f-be4c-4452-a1ec-4d43acbeaa15,6181f964-b4b1-473b-b2e2-2cd74883864e
3,How do ChatGPT and other LLMs demonstrate thei...,"Alright, here’s the lowdown, straight fire sty...",None,The context explains that ChatGPT and other LL...,1,1,1,6.853814,aa27845f-e17f-451f-b48e-c9eff2f5ec94,9e8d3071-e451-4aa5-9f42-65e47fa77283
4,"Considering the concerns about ethics, trainin...","Yo, here’s the lowdown straight from the conte...",None,The context emphasizes that LLMs face signific...,1,1,1,5.164886,c8efb7a9-51c2-46ae-9ce8-4b09c6d2c7c5,7ba06af1-1752-488b-b647-9b3485d2c1fa
5,How does the use of synthetic training data in...,"Alright, here we go—breaking it down smooth an...",None,The context explains that synthetic training d...,1,1,1,4.432002,b90d1d6e-d535-48ce-b22d-8fdf09ffac64,630ee6c9-f416-49ec-91a3-986939aa7a42
6,How have advancements in Large Language Models...,"Yo, check this out! In 2024, LLMs leveled up b...",None,"In 2024, significant advancements in Large Lan...",1,1,1,3.658089,61554441-1dbf-4ec3-87ae-dcc9100a64b6,59e018b4-4f02-446b-ba14-954bd7b1d991
7,How does Simon Willison’s 2023 weblog reflect ...,"Yo, Simon Willison’s 2023 weblog drops some se...",None,Simon Willison’s 2023 weblog highlights that 2...,1,1,1,7.542983,391fc1ee-0145-49ea-8ef6-66e887d3d4f5,9ae63278-5934-4a93-b2ab-b93ca7146d75
8,Who is Tom Scott and why is he important in th...,"Yo, I looked through the context, but there’s ...",None,Tom Scott is mentioned in the context as a per...,0,0,0,2.356796,4cf91cd7-b54d-47bf-af15-197c2ff3bab7,cf3cad7b-aa58-48d4-a118-02caeafc3d79
9,What is the significance of AI in 2023 accordi...,"Yo, 2023 was the year AI, specifically Large L...",None,Simon Willison states that 2023 was the breakt...,1,1,1,3.129303,52ef066f-86e8-4431-84e9-6113cbd9968e,3ccb600f-8ede-464f-ab73-d4578d69ddb8


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.